In [1]:
%pip install statsmodels
%pip install mlxtend

  Using cached statsmodels-0.13.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.9 MB)
  Using cached patsy-0.5.3-py2.py3-none-any.whl (233 kB)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
insurance_df = pd.read_csv("insurance.csv")
insurance_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
from sklearn.model_selection import train_test_split
X = insurance_df[['bmi', 'age', 'children']]
y = insurance_df['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
import statsmodels.api as sm
X_train = sm.add_constant(X_train)
linear_reg_model = sm.OLS(y_train, X_train)
linear_reg_model = linear_reg_model.fit()
print(linear_reg_model.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.110
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     43.86
Date:                Thu, 05 Jan 2023   Prob (F-statistic):           9.94e-27
Time:                        19:52:13   Log-Likelihood:                -11507.
No. Observations:                1070   AIC:                         2.302e+04
Df Residuals:                    1066   BIC:                         2.304e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6118.0462   1968.214     -3.108      0.0

In [5]:
import numpy as np
churn_df = pd.read_csv("telco_churn.csv")
churn_df.head()
churn_df['Churn'] = np.where(churn_df['Churn']=='Yes', 1, 0)

In [6]:
X = churn_df[['tenure', 'MonthlyCharges']]
X.loc[:,'tenure_squared'] = [x**2 for x in list(churn_df['tenure'])]
y = churn_df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [7]:
X_train = sm.add_constant(X_train)
log_reg_model = sm.Logit(y_train, X_train)
log_reg_model = log_reg_model.fit()
print(log_reg_model.summary())

Optimization terminated successfully.
         Current function value: 0.458122
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                 5634
Model:                          Logit   Df Residuals:                     5630
Method:                           MLE   Df Model:                            3
Date:                Thu, 05 Jan 2023   Pseudo R-squ.:                  0.2084
Time:                        19:52:13   Log-Likelihood:                -2581.1
converged:                       True   LL-Null:                       -3260.7
Covariance Type:            nonrobust   LLR p-value:                1.923e-294
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.6265      0.102    -16.021      0.000      -1.826      -1.428
tenure           

In [8]:
from sklearn.ensemble import RandomForestRegressor
X = insurance_df[['bmi', 'age', 'children']]
y = insurance_df['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model1 = RandomForestRegressor(random_state=42)
rf_model1.fit(X_train, y_train)
y_pred1 = rf_model1.predict(X_test)

rf_model2 = RandomForestRegressor(n_estimators= 50, max_depth=50, random_state=42)
rf_model2.fit(X_train, y_train)
y_pred2 = rf_model2.predict(X_test)


In [11]:
from mlxtend.evaluate import paired_ttest_5x2cv
_, p_value = paired_ttest_5x2cv(estimator1=rf_model1, estimator2=rf_model2, scoring='neg_mean_squared_error', X=X_train, y=y_train, random_seed=42)
if p_value < 0.05:
    print(f"P-value of {p_value} give evidence that model difference is significant")
else:
    print(f"P-value of {p_value} give evidence that model difference is not significant")

P-value of 0.33466026017280154 give evidence that model difference is not significant


In [19]:
from sklearn.linear_model import LinearRegression
lr_model1 = LinearRegression()
lr_model1.fit(X_train, y_train)
_, p_value = paired_ttest_5x2cv(estimator1=rf_model1, estimator2=lr_model1, scoring='neg_mean_squared_error', X=X_train, y=y_train, random_seed=42)
if p_value < 0.05:
    print(f"P-value of {p_value} give evidence that model difference is significant")
else:
    print(f"P-value of {p_value} give evidence that model difference is not significant")

P-value of 0.002834207433757008 give evidence that model difference is significant


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=434b0236-878d-4de0-9743-ad548853111f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>